In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/main/notebooks/samples/optimizer/ai_platform_optimizer_multi_objective.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
        </a>
    </td>
    <td>
        <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/main/notebooks/samples/optimizer/ai_platform_optimizer_multi_objective.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">View on GitHub
        </a>
     </td>
</table>


# Optimizing multiple objectives

This tutorial demonstrates AI Platform Optimizer multi-objective optimization.

### Objective

The goal is to __`minimize`__ the objective metric:
   ```
   y1 = r*sin(theta)
   ```

and simultaneously __`maximize`__ the objective metric:
   ```
   y2 = r*cos(theta)
   ```

that you will evaluate over the parameter space:

   - __`r`__ in [0,1],

   - __`theta`__ in [0, pi/2]

### Costs

This tutorial uses billable components of Google Cloud:

* AI Platform Training
* Cloud Storage

Learn about [AI Platform Training
pricing](https://cloud.google.com/ai-platform/training/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


### PIP install packages and dependencies

Install additional dependencies not installed in the notebook environment.

- Use the latest major GA version of the framework.

In [ ]:
! pip install -U google-cloud
! pip install -U google-cloud-storage
! pip install -U requests

#### Install tensorflow-cloud

Download and install tensorflow-cloud.

In [ ]:
! pip install -U tensorflow-cloud

In [ ]:
# Restart the kernel after pip installs
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project.](https://console.cloud.google.com/cloud-resource-manager)

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com)

4. If running locally on your own machine, you will need to install the [Google Cloud SDK](https://cloud.google.com/sdk).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Authenticate your Google Cloud account

**If you are using [AI Platform Notebooks](https://cloud.google.com/ai-platform/notebooks/docs/)**, your environment is already
authenticated. Skip these steps.

In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

# If you are running this tutorial in a notebook locally, replace the string
# below with the path to your service account key and run this cell to
# authenticate your Google Cloud account.
else:
    %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

# Log in to your account on Google Cloud
!gcloud auth login

### Import libraries and define constants

In [ ]:
import datetime
import json

from tensorflow_cloud.tuner import tuner, utils

## Tutorial


### Setup
This section defines some parameters and util methods to call AI Platform Optimizer APIs. Please fill in the following information to get started.

In [ ]:
# Fill in your username and project ID
USER = "[user-id]"  # @param {type: 'string'}
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# These will be automatically filled in.
STUDY_ID = "{}_study_{}".format(
    USER.replace("-", ""), datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
)  # @param {type: 'string'}
REGION = "us-central1"
! gcloud config set project $PROJECT_ID

### Create the study configuration

The following is a sample study configuration, built as a hierarchical python dictionary. It is already filled out. Run the cell to configure the study.

In [ ]:
# Parameter Configuration
param_r = {
    "parameter": "r",
    "type": "DOUBLE",
    "double_value_spec": {"min_value": 0, "max_value": 1},
}

param_theta = {
    "parameter": "theta",
    "type": "DOUBLE",
    "double_value_spec": {"min_value": 0, "max_value": 1.57},
}

# Objective Metrics
metric_y1 = {"metric": "y1", "goal": "MINIMIZE"}

metric_y2 = {"metric": "y2", "goal": "MAXIMIZE"}

# Put it all together in a study configuration
study_config = {
    "algorithm": "ALGORITHM_UNSPECIFIED",  # Let the service choose the `default` algorithm.
    "parameters": [
        param_r,
        param_theta,
    ],
    "metrics": [
        metric_y1,
        metric_y2,
    ],
}

print(json.dumps(study_config, indent=2, sort_keys=True))

### Create the study

Next, create the study, which you will subsequently run to optimize the two objectives.

In [ ]:
service = tuner.optimizer_client.create_or_load_study(
    PROJECT_ID, REGION, STUDY_ID, study_config
)

### Metric evaluation functions

Next, define some functions to evaluate the two objective metrics.

In [ ]:
import math


# r * sin(theta)
def Metric1Evaluation(r, theta):
    """Evaluate the first metric on the trial."""
    return r * math.sin(theta)


# r * cos(theta)
def Metric2Evaluation(r, theta):
    """Evaluate the second metric on the trial."""
    return r * math.cos(theta)


def CreateMetrics(trial_id, params):
    print(("=========== Start Trial: [{}] =============").format(trial_id))

    r = params["r"]
    theta = params["theta"]

    # Evaluate both objective metrics for this trial
    y1 = Metric1Evaluation(r, theta)
    y2 = Metric2Evaluation(r, theta)
    print(
        "[r = {}, theta = {}] => y1 = r*sin(theta) = {}, y2 = r*cos(theta) = {}".format(
            r, theta, y1, y2
        )
    )
    metric1 = {"metric": "y1", "value": y1}
    metric2 = {"metric": "y2", "value": y2}

    # Return the results for this trial
    return [
        metric1,
        metric2,
    ]

### Set configuration parameters for running trials

__`client_id`__ - The identifier of the client that is requesting the suggestion. If multiple SuggestTrialsRequests have the same `client_id`, the service will return the identical suggested trial if the trial is `PENDING`, and provide a new trial if the last suggested trial was completed.

__`suggestion_count_per_request`__ - The number of suggestions (trials) requested in a single request.

__`max_trial_id_to_stop`__ - The number of trials to explore before stopping. It is set to 4 to shorten the time to run the code, so don't expect convergence. For convergence, it would likely need to be about 20 (a good rule of thumb is to multiply the total dimensionality by 10).


In [ ]:
client_id = "client1"  # @param {type: 'string'}
suggestion_count_per_request = 5  # @param {type: 'integer'}
max_trial_id_to_stop = 4  # @param {type: 'integer'}

print("client_id: {}".format(client_id))
print("suggestion_count_per_request: {}".format(suggestion_count_per_request))
print("max_trial_id_to_stop: {}".format(max_trial_id_to_stop))

### Run AI Platform Optimizer trials

Run the trials.

In [ ]:
trial_id = 0
while trial_id < max_trial_id_to_stop:
    suggestions = service.get_suggestions(client_id, suggestion_count_per_request)
    for suggested_trial in suggestions["trials"]:
        trial_id = int(utils.get_trial_id(suggested_trial))

        trial = service.get_trial(trial_id)
        if trial["state"] in ["COMPLETED", "INFEASIBLE"]:
            continue

        # Parses the suggested parameters.
        params = utils.convert_optimizer_trial_to_dict(suggested_trial)

        service.report_intermediate_objective_value(
            step=1,
            elapsed_secs=0,
            metric_list=CreateMetrics(trial_id, params),
            trial_id=trial_id,
        )
        service.complete_trial(trial_id, trial_infeasible=False)

### [EXPERIMENTAL] Visualize the result

This section provides a module to visualize the trials for the above study.


In [ ]:
max_trials_to_annotate = 20

import matplotlib.pyplot as plt

trial_ids = []
y1 = []
y2 = []
trials = service.list_trials()
for trial in trials:
    if "finalMeasurement" in trial:
        trial_ids.append(int(trial["name"].split("/")[-1]))
        metrics = trial["finalMeasurement"]["metrics"]
        try:
            y1.append([m for m in metrics if m["metric"] == "y1"][0]["value"])
            y2.append([m for m in metrics if m["metric"] == "y2"][0]["value"])
        except Exception:
            pass

fig, ax = plt.subplots()
ax.scatter(y1, y2)
plt.xlabel("y1=r*sin(theta)")
plt.ylabel("y2=r*cos(theta)")
for i, trial_id in enumerate(trial_ids):
    # Only annotates the last `max_trials_to_annotate` trials
    if i > len(trial_ids) - max_trials_to_annotate:
        try:
            ax.annotate(trial_id, (y1[i], y2[i]))
        except Exception:
            pass
plt.gcf().set_size_inches((16, 16))

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.